## Wave Collapse Simulation
    

In [ ]:
# Updating disk so that we capture all angles and positions in order to collapse the superposition.
class Disk:
    def __init__(self, start, goal):
        self.x, self.y = tuple(element + 0.5 for element in start)
        self.goal = goal
        self.angle = random.uniform(0, 2 * np.pi)
        self.path = [(self.x, self.y, self.angle)]
        self.eye_memory = {}  # { (x, y): [list of angles over time] }

    def move(self, direction):
        dx, dy = direction
        self.x += dx
        self.y += dy
        self.angle += random.uniform(-90, 90) * np.pi / 180
        pos = (int(self.x), int(self.y))
        if pos not in self.eye_memory:
            self.eye_memory[pos] = []
        self.eye_memory[pos].append(self.angle)
        self.path.append((self.x, self.y, self.angle))

In [ ]:
# When we select point B2 at point C
def collapse_wave_function(disk, target_point):
    """ Retrocausally update eye direction distributions to favor B2 """
    for (x, y), angles in disk.eye_memory.items():
        if len(angles) < 2:
            continue

        # Current average eye direction at this past point
        current_mean = np.mean(angles)

        # Vector from this point to B2
        dx = target_point[0] + 0.5 - x
        dy = target_point[1] + 0.5 - y
        desired_angle = np.arctan2(dy, dx)

        # Softly adjust all stored angles toward desired_angle
        adjusted_angles = []
        for a in angles:
            delta = desired_angle - a
            a_new = a + 0.2 * delta  # 0.2 = retrocausal softness factor
            adjusted_angles.append(a_new)

        disk.eye_memory[(x, y)] = adjusted_angles

In [ ]:
# Visualisation
def display_grid_with_wave_collapse(disk, grid):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(grid.grid.T, origin='lower', cmap='gray', alpha=0.5)

    for (x, y), angles in disk.eye_memory.items():
        mean_angle = np.mean(angles)
        disk_patch = Circle((x + 0.5, y + 0.5), radius=DISK_RADIUS, edgecolor='purple', facecolor='blue', lw=1)
        ax.add_patch(disk_patch)

        line_x = x + 0.5 + (DISK_RADIUS - EYE_LENGTH_NEG) * np.cos(mean_angle)
        line_y = y + 0.5 + (DISK_RADIUS - EYE_LENGTH_NEG) * np.sin(mean_angle)
        ax.plot([x + 0.5, line_x], [y + 0.5, line_y], color='gold', lw=1)

    ax.set_xlim(0, GRID_SIZE)
    ax.set_ylim(0, GRID_SIZE)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('equal')
    plt.title("Collapsed Eye States Favoring B2")
    plt.show()

In [ ]:
def collapse_wave_function_with_observers(disk, collapse_point, observer_strength=0.3):
    """
    Performs a 'soft' retrocausal adjustment of past eye directions toward the collapse_point.
    This simulates the wave-function collapse affecting perception history.
    """
    updated_path = []
    for x, y, old_angle in disk.path:
        # Direction from (x, y) to collapse_point
        target_dx = collapse_point[0] - x
        target_dy = collapse_point[1] - y
        target_angle = np.arctan2(target_dy, target_dx)
        
        # Interpolate angle toward the target_angle
        delta_angle = (target_angle - old_angle + np.pi) % (2 * np.pi) - np.pi
        new_angle = old_angle + observer_strength * delta_angle
        
        updated_path.append((x, y, new_angle))
    
    disk.path = updated_path


collapse_point = (25, 25)  # ← Your chosen "B2" endpoint
collapse_wave_function_with_observers(disk, collapse_point, observer_strength=0.3)

